# 4444

## What's new:

1- Using scipy for labeling 2


In [45]:
import numpy as np
import pandas as pd
np.random.seed(42)   # reproducibility

In [66]:
# ================================================================
# PART 1: Create random dataset (10 rows) and print forecast_class
# ================================================================
predicted_df = pd.DataFrame({
    "DATETIME": pd.date_range("2025-08-13 22:00:00", periods=10, freq="h"),
    "forecast_class": np.random.choice([0, 1, 2], size=10),
    "prob_0": np.random.rand(10),
    "prob_1": np.random.rand(10),
    "prob_2": np.random.rand(10),
})


vals = predicted_df["forecast_class"].tolist()
print("[" + " ".join(str(x) for x in vals) + "]")

# ================================================================
# PART 2: Filtering rules
# ================================================================

def filter_forecast_classes(fc):
    fc = fc.copy()
    n = len(fc)

    # ------------------------------------------
    # Rule A: Keep only last in consecutive runs
    # ------------------------------------------
    i = 0
    while i < n:
        if fc[i] in [1, 2]:
            val = fc[i]
            start = i
            end = i
            while end + 1 < n and fc[end + 1] == val:
                end += 1
            # Keep only the last occurrence
            for k in range(start, end):
                fc[k] = 0
            i = end
        i += 1

    # ---------------------------------------------------------
    # Rule B & C: If val (1 or 2) appears, then zeros, then val
    # ---------------------------------------------------------
    for val in [1, 2]:
        last_pos = None
        for i in range(n):
            if fc[i] == val:
                if last_pos is not None:
                    # Remove the old one (set to 0)
                    fc[last_pos] = 0
                last_pos = i
            elif fc[i] != 0:
                # reset tracking when 0-block is broken by other class
                last_pos = None

    return fc



filtered = filter_forecast_classes(predicted_df["forecast_class"].tolist())
print("[" + " ".join(str(x) for x in filtered) + "]")


[1 1 1 2 2 0 1 0 0 0]
[0 0 1 0 2 0 1 0 0 0]


In [54]:
predicted_df["forecast_class_filtered"] = filtered
predicted_df